## update user contributions


In [1]:
## should do this on a monthly basis because 1) this is will quickly use up BULK API's allotted quantity
## and 2) db2 only updates some usage numbers on a yearly or monthly basis

## currently this script takes over 25 minutes to upload to SF

In [2]:
# API settings
api_url = '/services/data/v43.0/sobjects'
external_id = 'Name'
#object_id = 'ContactToolAssociation__c'

# login parameters to be handled by Papermill
sql_login_params = {"username": "wang2506_ro", "password": "fnVnwcCS7iT45EsA"}
sf_login_params = {
    "grant_type": "password",
    "client_id": "3MVG95jctIhbyCppj0SNJ75IsZ1y8UPGZtSNF4j8FNVXz.De8Lu4jHm3rjRosAtsHy6qjHx3i4S_QbQzvBePG",
    "client_secret": "D1623C6D3607D4FC8004B92C761DFB6C1F70CCD129C5501E357028DFA00F5764",
    "username":"wang2506@purdue.edu",
    "password":"sf2021shitOPmlIiFMLnrudgC6oSX0WV1T",     
}
    #"username": "wang159-4j1v@force.com",
    #"password": "napoleon0eZ3PQpQqE3C3z4wWqEqKGhQ8",

In [3]:
import sys
#sys.path.append('/home/users/wang2506/nanohub_salesforce_integ/salesforce')

import pandas as pd
import time
import datetime

from DB2SalesforceAPI import DB2SalesforceAPI

## Obtain information from DB2 

In [16]:
#pre-filtering queries using datetime
date_temp = datetime.datetime.today()
year = date_temp.year
month = date_temp.month
period = 14
if month < 10:
    date_str0 = '\''+ str(year)+'-0'+str(month)+'-00'+'\''
    date_str1 = '\''+str(year)+'-0'+str(month)+'-01'+'\''
else:
    date_str0 = '\''+ str(year)+'-'+str(month)+'-00'+'\''
    date_str1 = '\''+str(year)+'-'+str(month)+'-01'+'\''

In [17]:
print(date_str0)
print(date_str1)

'2020-09-00'
'2020-09-01'


In [7]:
# connect with DB2
import sqlalchemy as sql

engine = sql.create_engine('mysql+pymysql://%s:%s@127.0.0.1/nanohub' \
                                               %(sql_login_params['username'], sql_login_params['password']))

In [7]:
# extract the wiki contributions
sql_query = "select hits, created_by from jos_wiki_pages where state = '1' and scope = 'site';"
wiki_results_df = pd.read_sql_query(sql_query,engine)
#corresponds to wiki_usage__c, wiki_contributions__c

In [8]:
# extract paper contributions; need investigate how to get the usage per paper
sql_query = "select authorid, hits from jos_resources left join jos_author_assoc "\
            "on jos_author_assoc.subid = jos_resources.id where type = '3' and access in ('0','3','1') "\
            "and standalone = '1' and published in ('1');"
papers_df = pd.read_sql_query(sql_query,engine)

In [9]:
# extract presentation contributions; need to see how to get usage per presentation
sql_query = "select authorid, hits from jos_resources left join jos_author_assoc "\
            "on jos_author_assoc.subid = jos_resources.id where type = '1' and access in ('0','3','1') "\
            "and standalone = '1' and published in ('1');"
presentation_df = pd.read_sql_query(sql_query,engine)

In [10]:
# extract presentation contributions; need to see how to get usage per presentation
sql_query = "select authorid, hits from jos_resources left join jos_author_assoc "\
            "on jos_author_assoc.subid = jos_resources.id where type = '7' and access in ('0','3','1') "\
            "and standalone = '1' and published in ('1');"
tools_df = pd.read_sql_query(sql_query,engine)

In [21]:
# getting the users that used the contributions
sql_query = "select * from jos_author_stats WHERE datetime " \
    "between " + date_str0 +" and " + date_str1 + " and period = 14;"

result_users_df = pd.read_sql_query(sql_query, engine)

In [12]:
display(wiki_results_df.head(2))
display(papers_df.head(2))
display(presentation_df.head(2))
display(tools_df.head(2))
display(result_users_df.head(2))
print(result_users_df.shape)

,hits,created_by
0,6540,4843
1,44525,8780


,authorid,hits
0,1871.0,13674
1,2862.0,24552


,authorid,hits
0,-16.0,10468
1,3013.0,6991


,authorid,hits
0,-5.0,251605
1,8629.0,50032


,id,authorid,tool_users,andmore_users,total_users,datetime,period
0,1149593,3089,531.0,6078,6497,2020-09-00 00:00:00,14
1,1149596,4830,531.0,7351,7796,2020-09-00 00:00:00,14


(3187, 7)


In [13]:
## get rid of all negative authorids from all of the dataframes
wikidrops = []
wikikeeps = []
for ind,val in enumerate(wiki_results_df['created_by'].to_list()):
    if val <= 0:
        wikidrops.append(ind)
    else:
        wikikeeps.append(ind)
wiki_results_df = wiki_results_df.iloc[wikikeeps,:]

In [14]:
display(wiki_results_df.head(2))

,hits,created_by
0,6540,4843
1,44525,8780


In [15]:
## papers df 
pdrops = []
pkeeps = []
for ind,val in enumerate(papers_df['authorid'].to_list()):
    if val <= 0:
        pdrops.append(ind)
    else:
        pkeeps.append(ind)
papers_df = papers_df.iloc[pkeeps,:]

In [16]:
display(papers_df.head(2))

,authorid,hits
0,1871.0,13674
1,2862.0,24552


In [17]:
## presentation_df
prdrops = []
prkeeps = []
for ind,val in enumerate(presentation_df['authorid'].to_list()):
    if val <= 0:
        prdrops.append(ind)
    else:
        prkeeps.append(ind)
presentation_df = presentation_df.iloc[prkeeps,:]

In [18]:
display(presentation_df.head(2))

,authorid,hits
1,3013.0,6991
2,2862.0,21358


## roll up information from wiki_results_df, papers_df, and presentation_df

In [75]:
# wiki_results_df

## get the number of unique wiki contributions
wikic = wiki_results_df.groupby(['created_by']).size().reset_index() #need reset_index to save the authorid/created_by field
wikic.columns = ['nanoHUB_user_ID__c','wiki_contributions__c']
wikic = pd.DataFrame(wikic)

## rollup usage
wikic2 = wiki_results_df.groupby(['created_by']).sum().reset_index()
wikic2.columns = ['nanoHUB_user_ID__c','wiki_usage__c']
wikic2 = pd.DataFrame(wikic2)

wikic = wikic.join(wikic2,lsuffix='1', rsuffix='2')
display(wikic.head(2))

,nanoHUB_user_ID__c1,wiki_contributions__c,nanoHUB_user_ID__c2,wiki_usage__c
0,1852,3,1852,27718
1,2862,1,2862,5280


In [78]:
wikic = wikic.drop(columns=['nanoHUB_user_ID__c2'])
wikic = wikic.rename(columns={"nanoHUB_user_ID__c1": "nanoHUB_user_ID__c"})
display(wikic.head(2))

,nanoHUB_user_ID__c,wiki_contributions__c,wiki_usage__c
0,1852,3,27718
1,2862,1,5280


In [79]:
# papers_df - just need the number of unique contributions

## get the number of unique paper contributions
papersc = papers_df.groupby(['authorid']).size().reset_index() #need reset_index to save the authorid/created_by field
papersc.columns = ['nanoHUB_user_ID__c','paper_contributions__c']
papersc = pd.DataFrame(papersc)

## to add in the number of paper hits in the future ##TODO

display(papersc.head(2))

,nanoHUB_user_ID__c,paper_contributions__c
0,1852.0,3
1,1871.0,1


In [80]:
# presentation_df - just need the number of unique contributions

## get the number of unique presentation contributions
pres_c = presentation_df.groupby(['authorid']).size().reset_index() #need reset_index to save the authorid/created_by field
pres_c.columns = ['nanoHUB_user_ID__c','presentations_contributions__c']
pres_c = pd.DataFrame(pres_c)

## to add in the number of paper hits in the future ##TODO

display(pres_c.head(2))

,nanoHUB_user_ID__c,presentations_contributions__c
0,1742.0,2
1,1810.0,1


## Big picture users dataframe development

In [94]:
# rename the authorid in result_users_df, then join all of the dataframes
ovr_df = result_users_df.rename(columns={"authorid": "nanoHUB_user_ID__c"})
ovr_df['contributor__c'] = 0

In [95]:
## get rid of all negative nanohub ids
drops = []
keeps = []
for ind,val in enumerate(ovr_df['nanoHUB_user_ID__c'].to_list()):
    if val <= 0:
        drops.append(ind)
    else:
        keeps.append(ind)
ovr_df = ovr_df.iloc[keeps,:]

ovr_df = ovr_df.drop(columns=['datetime','period','id'])

display(ovr_df.head(2))
display(ovr_df.tail(2))

,nanoHUB_user_ID__c,tool_users,andmore_users,total_users,contributor__c
0,3089,531.0,6078,6497,0
1,4830,531.0,7351,7796,0


,nanoHUB_user_ID__c,tool_users,andmore_users,total_users,contributor__c
3185,288481,0.0,0,0,0
3186,277421,0.0,0,0,0


In [96]:
ovr_df = ovr_df.set_index('nanoHUB_user_ID__c').join(wikic.set_index('nanoHUB_user_ID__c')) 
ovr_df = ovr_df.join(papersc.set_index('nanoHUB_user_ID__c')) 
ovr_df = ovr_df.join(pres_c.set_index('nanoHUB_user_ID__c')) 

display(ovr_df.head(10))
#use .reset_index() to extract it later

,tool_users,andmore_users,total_users,contributor__c,wiki_contributions__c,wiki_usage__c,paper_contributions__c,presentations_contributions__c
nanoHUB_user_ID__c,,,,,,,,
3089,531.0,6078,6497,0,NaN,NaN,2.0,1.0
4830,531.0,7351,7796,0,NaN,NaN,3.0,1.0
5827,NaN,2455,2455,0,NaN,NaN,NaN,1.0
8629,5781.0,187246,190843,0,1.0,44764.0,12.0,244.0
9736,44735.0,789346,813908,0,6.0,40012.0,4.0,39.0
3013,31568.0,14887,44747,0,2.0,22898.0,1.0,26.0
2053,2702.0,3945,6206,0,NaN,NaN,2.0,1.0
10740,NaN,1722,1722,0,NaN,NaN,NaN,1.0
8690,NaN,585,585,0,NaN,NaN,NaN,NaN


## Get the current nanohub user IDs and their tool counts from SF
then begin the dataframe merge and post-processing

In [98]:
# create DB2 to Salesforce API object
db_s = DB2SalesforceAPI(sf_login_params)

Obtained Salesforce access token ...... True


In [99]:
# get nanohub ID for contacts to being matching and processing
sf_username_df = db_s.query_data('SELECT nanoHUB_user_ID__c,Total_tool_contributions__c FROM Contact')

# display
display(sf_username_df.head(2))
print(sf_username_df.shape)

[Success] Bulk job creation successful. Job ID = 7505w00000QMtt6AAD
{"id":"7505w00000QMtt6AAD","operation":"query","object":"Contact","createdById":"0055w00000DM5bOAAT","createdDate":"2020-09-25T20:34:59.000+0000","systemModstamp":"2020-09-25T20:34:59.000+0000","state":"InProgress","concurrencyMode":"Parallel","contentType":"CSV","apiVersion":47.0,"jobType":"V2Query","lineEnding":"LF","columnDelimiter":"COMMA","numberRecordsProcessed":0,"retries":0,"totalProcessingTime":0}
{"id":"7505w00000QMtt6AAD","operation":"query","object":"Contact","createdById":"0055w00000DM5bOAAT","createdDate":"2020-09-25T20:34:59.000+0000","systemModstamp":"2020-09-25T20:34:59.000+0000","state":"InProgress","concurrencyMode":"Parallel","contentType":"CSV","apiVersion":47.0,"jobType":"V2Query","lineEnding":"LF","columnDelimiter":"COMMA","numberRecordsProcessed":0,"retries":0,"totalProcessingTime":0}
{"id":"7505w00000QMtt6AAD","operation":"query","object":"Contact","createdById":"0055w00000DM5bOAAT","createdDat

,Total_tool_contributions__c,nanoHUB_user_ID__c
0,0.0,998.0
1,0.0,1683.0


(226698, 2)


In [100]:
# join the two dataframes
nover_df = sf_username_df.set_index('nanoHUB_user_ID__c').join(ovr_df.set_index('nanoHUB_user_ID__c'))

In [101]:
display(nover_df.head(2))

,Total_tool_contributions__c,tool_users,andmore_users,total_users,contributor__c,wiki_contributions__c,wiki_usage__c,paper_contributions__c,presentations_contributions__c
nanoHUB_user_ID__c,,,,,,,,,
998.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1683.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [102]:
## calculate the non tool contributions
nover_df['And_More_Contributions__c'] = nover_df['wiki_contributions__c']+nover_df['paper_contributions__c']\
        +nover_df['presentations_contributions__c']

In [103]:
# recover nanohub ids 
nover_df = nover_df.reset_index()
display(nover_df.head(2))

,nanoHUB_user_ID__c,Total_tool_contributions__c,tool_users,andmore_users,total_users,contributor__c,wiki_contributions__c,wiki_usage__c,paper_contributions__c,presentations_contributions__c,And_More_Contributions__c
0,998.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1683.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [104]:
## remove NAN rows for index
nover_df = nover_df.dropna(subset=['nanoHUB_user_ID__c'])
nover_df.shape

(226662, 11)

In [105]:
## then make the indexes integers
import numpy as np
nover_df['nanoHUB_user_ID__c'] = nover_df['nanoHUB_user_ID__c'].apply(lambda x: int(x))
display(nover_df.head(2))

,nanoHUB_user_ID__c,Total_tool_contributions__c,tool_users,andmore_users,total_users,contributor__c,wiki_contributions__c,wiki_usage__c,paper_contributions__c,presentations_contributions__c,And_More_Contributions__c
0,998,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1683,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [106]:
# calculate contributor flag
for index,val in enumerate(nover_df['contributor__c'].to_list()):
    # if any of the entries are non zero/non-NaN, then contributor__c = 1
    if nover_df['total_users'][index] > 0:
        nover_df['contributor__c'][index] = 1
    elif nover_df['wiki_contributions__c'][index] > 0:
        nover_df['contributor__c'][index] = 1
    elif nover_df['paper_contributions__c'][index] > 0:
        nover_df['contributor__c'][index] = 1
    elif nover_df['presentations_contributions__c'][index] > 0:
        nover_df['contributor__c'][index] = 1
    elif nover_df['Total_tool_contributions__c'][index] > 0:
        nover_df['contributor__c'][index] = 1
        
display(nover_df.head(10))

C:\Users\henry\Anaconda3\envs\tf\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
C:\Users\henry\Anaconda3\envs\tf\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
C:\Users\henry\Anaconda3\envs\tf\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path(

,nanoHUB_user_ID__c,Total_tool_contributions__c,tool_users,andmore_users,total_users,contributor__c,wiki_contributions__c,wiki_usage__c,paper_contributions__c,presentations_contributions__c,And_More_Contributions__c
0,998,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1683,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1684,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1685,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1686,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,1687,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,1688,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,1689,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,1690,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,1691,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Fitting data to SF format

In [108]:
## creating and populating the sf dataframe
df_sf = pd.DataFrame()
temp_df = nover_df.fillna(0)

# populating df_sf and make integers
df_sf['nanoHUB_user_ID__c'] = temp_df['nanoHUB_user_ID__c'].apply(lambda x: int(x))
df_sf['And_More_Contributions__c'] = temp_df['And_More_Contributions__c'].apply(lambda x: int(x))
df_sf['contributed_tool_usage__c'] = temp_df['tool_users'].apply(lambda x: int(x))
df_sf['contributed_andmore_usage__c'] = temp_df['andmore_users'].apply(lambda x: int(x))
df_sf['contributor__c'] = temp_df['contributor__c']

df_sf['wiki_contributions__c'] = temp_df['wiki_contributions__c']
df_sf['wiki_usage__c'] = temp_df['wiki_usage__c']
df_sf['paper_contributions__c'] = temp_df['paper_contributions__c']
df_sf['presentations_contributions__c'] = temp_df['presentations_contributions__c']

display(df_sf.head(2))

,nanoHUB_user_ID__c,And_More_Contributions__c,contributed_tool_usage__c,contributed_andmore_usage__c,contributor__c,wiki_contributions__c,wiki_usage__c,paper_contributions__c,presentations_contributions__c
0,998,0,0,0,0.0,0.0,0.0,0.0,0.0
1,1683,0,0,0,0.0,0.0,0.0,0.0,0.0


## send data to SF

In [109]:
# create DB2 to Salesforce API object
db_s = DB2SalesforceAPI(sf_login_params)

# specify Salesforce object ID and external ID
db_s.object_id = 'Contact'
db_s.external_id = 'nanoHUB_user_ID__c'

Obtained Salesforce access token ...... True


In [110]:
# send data to Salesforce
db_s.send_data(df_sf)

[Success] Bulk job creation successful. Job ID = 7505w00000QMty1AAD
hello
[Success] CSV upload successful. Job ID = 7505w00000QMty1AAD
[Success] Closing job successful. Job ID = 7505w00000QMty1AAD


In [113]:
# check status
from pprint import pprint

pprint(db_s.check_bulk_status())

{'apexProcessingTime': 19841467,
 'apiActiveProcessingTime': 2041489,
 'apiVersion': 47.0,
 'columnDelimiter': 'COMMA',
 'concurrencyMode': 'Parallel',
 'contentType': 'CSV',
 'createdById': '0055w00000DM5bOAAT',
 'createdDate': '2020-09-25T20:40:44.000+0000',
 'externalIdFieldName': 'nanoHUB_user_ID__c',
 'id': '7505w00000QMty1AAD',
 'jobType': 'V2Ingest',
 'lineEnding': 'LF',
 'numberRecordsFailed': 0,
 'numberRecordsProcessed': 226662,
 'object': 'Contact',
 'operation': 'upsert',
 'retries': 0,
 'state': 'JobComplete',
 'systemModstamp': '2020-09-25T20:50:03.000+0000',
 'totalProcessingTime': 2260639}


In [114]:
pprint(db_s.check_bulk_failed_results())

'"sf__Id","sf__Error",And_More_Contributions__c,contributed_andmore_usage__c,contributed_tool_usage__c,contributor__c,nanoHUB_user_ID__c,paper_contributions__c,presentations_contributions__c,wiki_contributions__c,wiki_usage__c\n'
